In [3]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown,display,update_display
from openai import OpenAI

In [20]:
load_dotenv()
OPENAI_API = os.getenv('OPENAI_API')

if OPENAI_API and OPENAI_API[:8] == "sk-proj-":
    print("API KEY IS PERFECT")
else:
    print("Something went wrong")

openai = OpenAI(api_key=OPENAI_API)

MODEL = "gpt-4o-mini"

API KEY IS PERFECT


In [70]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [89]:
op = Website('https://anthropic.com/')

In [98]:
# print(op.get_contents())
print("="*20)
print(op.links)

['#main', '#footer', 'https://www.anthropic.com/', 'https://www.anthropic.com/claude', 'https://www.anthropic.com/team', 'https://www.anthropic.com/enterprise', 'https://www.anthropic.com/education', 'https://www.anthropic.com/pricing', 'https://claude.ai/download', 'https://claude.ai/', 'https://www.anthropic.com/news/claude-character', 'https://www.anthropic.com/api', 'https://docs.anthropic.com/', 'https://www.anthropic.com/pricing#api', 'https://console.anthropic.com/', 'https://docs.anthropic.com/en/docs/welcome', 'https://www.anthropic.com/solutions/agents', 'https://www.anthropic.com/solutions/coding', 'https://www.anthropic.com/solutions/customer-support', 'https://www.anthropic.com/customers', 'https://www.anthropic.com/research', 'https://www.anthropic.com/economic-index', 'https://www.anthropic.com/claude/opus', 'https://www.anthropic.com/claude/sonnet', 'https://www.anthropic.com/claude/haiku', 'https://www.anthropic.com/news/visible-extended-thinking', 'https://www.anthrop

In [56]:
system_prompt = """
You are provided with a list of links found on a webpage.\
You are able to decide which of the links would be the most relevant to include
in a brocher about a company if it is a IT company include things like careers,about,etc.. and 
if it is some shopping page add links like about, discounts, etc...\

You should respond in a JSON as in this example. \
Example:
{
    "links": [
        {"type: "about page,"url":"https://full_url/about"}
        {"type: "careers page","url":"https://full_url/careers"}
    ]
}
"""

In [57]:
print(system_prompt)


You are provided with a list of links found on a webpage.You are able to decide which of the links would be the most relevant to include
in a brocher about a company if it is a IT company include things like careers,about,etc.. and 
if it is some shopping page add links like about, discounts, etc...
You should respond in a JSON as in this example. Example:
{
    "links": [
        {"type: "about page,"url":"https://full_url/about"}
        {"type: "careers page","url":"https://full_url/careers"}
    ]
}



In [58]:
def user_prompt(website):
    user_prompt = f"""
    Here is the list of links of the company website of {website.url}
    Please decide which of these are relevant web links for a brocher about the company, respond with the full https URL.
    Do not include terms of service,privacy and emails. \
    \n
    {website.links}
    """
    return user_prompt

In [75]:
print(user_prompt(op))


    Here is the list of links of the company website of https://edwarddonner.com/
    Please decide which of these are relevant web links for a brocher about the company, respond with the full https URL.
    Do not include terms of service,privacy and emails.     

    ['https://edwarddonner.com/', 'https://edwarddonner.com/connect-four/', 'https://edwarddonner.com/outsmart/', 'https://edwarddonner.com/about-me-and-about-nebula/', 'https://edwarddonner.com/posts/', 'https://edwarddonner.com/', 'https://news.ycombinator.com', 'https://nebula.io/?utm_source=ed&utm_medium=referral', 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html', 'https://patents.google.com/patent/US20210049536A1/', 'https://www.linkedin.com/in/eddonner/', 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/', 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/', 'ht

In [86]:
def get_relevant_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role":"system","content":system_prompt},
            {"role":"user","content":user_prompt(website)}
        ],
        response_format={"type":"json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [88]:
get_relevant_links("https://anthropic.com/")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'events page', 'url': 'https://www.anthropic.com/events'},
  {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'},
  {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'},
  {'type': 'enterprise solutions page',
   'url': 'https://www.anthropic.com/enterprise'},
  {'type': 'education page', 'url': 'https://www.anthropic.com/education'}]}

In [ ]:
# Website('https://www.anthropic.com/education').get_contents()

In [103]:
def get_all_details(url):
    result = "Landing Page:\n"
    result += Website(url).get_contents()
    links = get_relevant_links(url)
    print("Found links:",links)
    for link in links['links']:
        result += f"\n\n{link['type']}\n"
        result += Website(link['url']).get_contents()
    return result

In [104]:
get_all_details("https://anthropic.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}, {'type': 'contact sales page', 'url': 'https://www.anthropic.com/contact-sales'}]}


"Landing Page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nChat with Claude\nOverview\nTeam plan\nEnterprise plan\nEducation plan\nExplore pricing\nDownload apps\nClaude log in\nNews\nClaude’s character\nAPI\nBuild with Claude\nAPI\xa0overview\nDeveloper docs\nExplore pricing\nConsole log in\nNews\nLearn how to build with Claude\nSolutions\nCollaborate with Claude\nAI\xa0agents\nCoding\nCustomer support\nCase studies\nHear from our customers\nResearch\nResearch\nOverview\nEconomic Index\nClaude model family\nClaude Opus 4\nClaude Sonnet 4\nClaude Haiku 3.5\nResearch\nClaude’s extended thinking\nCommitments\nInitiatives\nTransparency\nResponsible scaling policy\nTrust center\nSecurity and compliance\nAnnouncement\nISO\xa042001 certification\nLearn\nLearning resources\nCustomer stories\nEngineering at Anthropic\nAnthropic Academy\nCompany\nAbout\nCareers\nEvents\nEngineering\nBuilding effective agents\nNews\nEN\nThis is some text i

In [105]:
system_prompt2 = """
You are an assistant that analyzes the contents of several relevant pages of a company website.\
Create a short brochure about the company for custumors, investors and recruitors. Respond in Markdown.\
Include details of the company culture, customers and careers/jobs if you have the information. 
"""

In [106]:
def get_brochure_user_prompt(company_name,url):
    user_prompt = f"You are looking at the company called {company_name}"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the cmopany"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:10_000]
    return user_prompt

In [107]:
get_brochure_user_prompt("Anthropic","https://anthropic.com")

Found links: {'links': [{'type': 'home page', 'url': 'https://www.anthropic.com/'}, {'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'press releases/news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}, {'type': 'support page', 'url': 'https://support.anthropic.com/'}]}


'You are looking at the company called AnthropicHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the cmopanyLanding Page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nChat with Claude\nOverview\nTeam plan\nEnterprise plan\nEducation plan\nExplore pricing\nDownload apps\nClaude log in\nNews\nClaude’s character\nAPI\nBuild with Claude\nAPI\xa0overview\nDeveloper docs\nExplore pricing\nConsole log in\nNews\nLearn how to build with Claude\nSolutions\nCollaborate with Claude\nAI\xa0agents\nCoding\nCustomer support\nCase studies\nHear from our customers\nResearch\nResearch\nOverview\nEconomic Index\nClaude model family\nClaude Opus 4\nClaude Sonnet 4\nClaude Haiku 3.5\nResearch\nClaude’s extended thinking\nCommitments\nInitiatives\nTransparency\nResponsible scaling policy\nTrust center\nSecurity and compliance\nAnnouncement\nISO\xa042001 certification\nLearn\nLearning 

In [ ]:
def create_brochure(company_name,url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role":"system","content":system_prompt2},
            {"role":"user","content":get_brochure_user_prompt(company_name,url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [113]:
create_brochure("Anthropic","https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}, {'type': 'contact sales page', 'url': 'https://www.anthropic.com/contact-sales'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}]}


# Anthropic: Empowering the Future with Safe AI

Welcome to **Anthropic**, where we prioritize safety and reliability in AI development. Our commitment to fostering responsible AI systems drives everything we do, ensuring our technology creates positive long-term impacts on society.

## Company Overview

Anthropic is an AI safety and research company dedicated to building trustworthy AI systems. We focus on creating reliable, interpretable, and steerable technologies that serve humanity’s needs. Through innovative research and our state-of-the-art models, including **Claude Opus 4** and **Claude Sonnet 4**, we redefine what AI can be.

### Our Purpose
At Anthropic, we understand the vast and transformative impact of AI on the world. Our mission is to explore the opportunities and risks posed by AI, designing systems that empower users and enhance operations. 

## Company Culture

### Collaborative Environment
Our interdisciplinary team is comprised of researchers, engineers, policy experts, and operational leaders from diverse backgrounds, including NASA, startups, and more. This richness of experience fosters a culture of cooperation and innovation, as we work together on evolving AI frontiers.

### Commitment to Safety
Safety is not just an objective—it's a cornerstone of our philosophy. We treat AI safety as a systematic science, integrating findings from our research to continuously improve our products while advocating for broader industry safety practices.

### Employee Development
At Anthropic, we value learning and growth, offering access to the **Anthropic Academy** where employees and partners can enhance their skills and knowledge related to our AI systems.

## Who We Serve

Anthropic's AI solutions cater to a myriad of sectors, including:

- **Businesses:** Enhancing productivity through AI-driven solutions.
- **Nonprofits:** Developing tools that can support social causes.
- **Education:** Offering learning platforms that empower users to build with **Claude**.

### Customer Stories
We take pride in our collaborative success. Our customers' testimonials emphasize the transformative effects of our tools on their businesses and operations, showcasing real-world applications of AI.

## Join Us

If you are excited about shaping the future of AI safety and innovation, Anthropic offers numerous **career opportunities**. We’re constantly looking for passionate individuals ready to make a difference in the AI landscape.

### Open Roles Include:
- AI Researcher
- Software Engineer
- Policy Analyst
- Operations Specialist

## Get in Touch

If you're a customer interested in implementing AI solutions, an investor looking to be part of a pioneering company, or a recruiter eager to discover potential talent, we invite you to explore our offerings.

- **Website:** [Anthropic.ai](https://www.anthropic.ai)
- **Contact Us:** For inquiries or to learn more about us and our products, please reach out on our website.

Join us in building a future where AI systems are not only intelligent but also safe and reliable. Welcome to Anthropic!